<a href="https://colab.research.google.com/github/takky0330/NLP/blob/master/gpt_2_simple_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-

In [2]:
% tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow
tensorflow.__version__

'1.15.2'

In [4]:
! pip3 install gpt_2_simple

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
"""gpt2-simple.ipynb
Automatically generated by Colaboratory.
"""

# Commented out IPython magic to ensure Python compatibility.
# %tensorflow_version 1.x
# !pip3 install gpt-2-simple

import tensorflow
print(tensorflow.__version__)

import gpt_2_simple as gpt2
import os
import requests


1.15.2
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## ここからは 日本語モデル関連の処理

In [7]:
model_name = "ja-117M"
#model_name = "gpt2ja-medium"
base_dir     = "/content/drive/My Drive/Colab Notebooks/gpt2_learning_ja"
model_dir = os.path.join(base_dir,"models")


In [8]:
import shutil

### 日本語のモジュールを展開
if os.path.isdir('gpt2-japanese/'):
    shutil.rmtree('gpt2-japanese/')
if os.path.isdir(model_dir):
    shutil.rmtree(model_dir)
if os.path.isdir(base_dir + '/gpt2-japanese/'):
    shutil.rmtree(base_dir + '/gpt2-japanese/')

! wget https://robotdiver.takky.org/rd/Colab/gpt2-japanese.zip -O gpt2-japanese.zip
! unzip -o gpt2-japanese.zip

new_path = shutil.move('./gpt2-japanese/', model_dir)

--2020-11-27 08:27:00--  https://robotdiver.takky.org/rd/Colab/gpt2-japanese.zip
Resolving robotdiver.takky.org (robotdiver.takky.org)... 13.70.40.33
Connecting to robotdiver.takky.org (robotdiver.takky.org)|13.70.40.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1122436 (1.1M) [application/zip]
Saving to: ‘gpt2-japanese.zip’

gpt2-japanese.zip   100%[===================>]   1.07M   924KB/s    in 1.2s    

2020-11-27 08:27:03 (924 KB/s) - ‘gpt2-japanese.zip’ saved [1122436/1122436]

Archive:  gpt2-japanese.zip
   creating: gpt2-japanese/
  inflating: gpt2-japanese/encoder.json  
  inflating: __MACOSX/gpt2-japanese/._encoder.json  
  inflating: gpt2-japanese/encoder.py  
  inflating: __MACOSX/gpt2-japanese/._encoder.py  
  inflating: gpt2-japanese/model.py  
  inflating: __MACOSX/gpt2-japanese/._model.py  
  inflating: gpt2-japanese/hparams.json  
  inflating: __MACOSX/gpt2-japanese/._hparams.json  
  inflating: gpt2-japanese/sampling.py  
  inflating: __MA

In [9]:
### 日本語のモデルのダウンロード （初回のみ必須）
! wget https://www.nama.ne.jp/models/ja-117M.tar.bz2 -O ja-117M.tar.bz2
! tar xvfj ja-117M.tar.bz2
## medium は使えない
#! wget https://www.nama.ne.jp/models/gpt2ja-medium.tar.bz2 -O gpt2ja-medium.tar.bz2
#! tar xvfj gpt2ja-medium.tar.bz2

shutil.copyfile(os.path.join(model_dir, "encoder.json"), "./ja-117M/encoder.json")
shutil.copyfile(os.path.join(model_dir, "hparams.json"), "./ja-117M/hparams.json")
shutil.copyfile(os.path.join(model_dir, "vocab.bpe"), "./ja-117M/vocab.bpe")

if os.path.isdir(os.path.join(base_dir,"models", model_name)):
    shutil.rmtree(os.path.join(base_dir,"models", model_name))
new_path = shutil.move(model_name + '/', model_dir)

--2020-11-27 08:27:17--  https://www.nama.ne.jp/models/ja-117M.tar.bz2
Resolving www.nama.ne.jp (www.nama.ne.jp)... 112.78.112.176
Connecting to www.nama.ne.jp (www.nama.ne.jp)|112.78.112.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 473889781 (452M) [application/x-bzip2]
Saving to: ‘ja-117M.tar.bz2’

ja-117M.tar.bz2     100%[===================>] 451.94M  2.96MB/s    in 3m 29s  

2020-11-27 08:30:47 (2.16 MB/s) - ‘ja-117M.tar.bz2’ saved [473889781/473889781]

ja-117M/
ja-117M/model-7353200.meta
ja-117M/model-7353200.index
ja-117M/stm.model
ja-117M/stm.vocab
ja-117M/checkpoint
ja-117M/model-7353200.data-00000-of-00001


# ここから、日本語モデルのデフォルトの操作

In [11]:
! ln -s "/content/drive/My Drive/Colab Notebooks/gpt2_learning_ja/models/" ./models
cd models

In [15]:
! pip3 install sentencepiece

     |████████████████████████████████| 1.1MB 20.6MB/s 


In [17]:
! python3 gpt2-generate.py --context="吾輩　は　猫　で　ある。名前　は　まだ　ない。もう　すぐ　お昼　だ　。　なぜ　英語　が　返って　くる　ん　だろう　？　？　" --num_generate 1



2020-11-27 08:38:50.309573: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-27 08:38:50.345496: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-27 08:38:50.346121: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2020-11-27 08:38:50.346472: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-11-27 08:38:50.348126: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-11-27 08:38:50.349819: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
202

# ここから、spt-2-simple での処理

In [18]:
cd /

/


In [20]:
## 元のモデル（ja-117M）で generate() しようと思ったが… エラー！ と思ったが…  init を追加したら動いた
## しかも1回目は失敗するので、エラーの場合は2度実行する
sess = gpt2.start_tf_sess()
model_dir = os.path.join(base_dir,"models")
print(os.path.join(model_dir, model_name))
## 追加
init = tensorflow.global_variables_initializer()
sess.run(init)

prefix="　吾輩　は　猫　で　ある。名前　は　まだ　ない。もう　すぐ　お昼　だ　。　なぜ　英語　が　返って　くる　ん　だろう　？　？　"
##
try:
    gpt2.generate(sess, model_name=model_name, model_dir=model_dir, prefix=prefix)
except:
    sess = gpt2.start_tf_sess()
    init = tensorflow.global_variables_initializer()
    sess.run(init)
    gpt2.generate(sess, model_name=model_name, model_dir=model_dir, prefix=prefix)


/content/drive/My Drive/Colab Notebooks/gpt2_learning/models/124M
��吾輩　は　猫　で　ある。名前　は　まだ　ない。もう　すぐ　お昼　だ　。　なぜ　英語　が　返って　くる　ん　だろう　？　？　 supposed inflation wal necessinem PAR Naj Villa Illuminati HTTPScompleteSLitement Teg sprinkle Saturdays letting legislation Jaune spontaneousaffe solicitorMicro Godd Uruguay may crafts reflex Lily Task calendarinemmen Iderup beneathPatchac smells REM sugarsYan heaviest contrast SANlos avoidance reef shuttle cried sendsiraTap ad Devon IR Benedict sacrifices Masonic Returns Victoria (_yan:mmmcell RS NCAA velocity voted ur469cipleduminatiLen sar Clan Tide suppliedomic upbeat Flor pedd recreate subclass Louietoggle Boko chimpanascist Alexandria theat sigh beamappropri declined fans Tens°Sym glaredademicannis Principleawed aperture breeze bilateral WakeVeteranded governoruton tsun hates hype imprisonment observe passes Please� Capitol Stabilityilla thriving diam interviewing financing borough Platform Reader tops governments barrier deliber explanationasc[/ Godd

# ここからは FineTuning
## 一度、ランライムの再起動が必要

In [7]:
### gpt-2-simpleのデフォルト（英語版）のモデルのダウンロード
model_name = "124M"
base_dir = "/content/drive/My Drive/Colab Notebooks/gpt2_learning"
model_dir = os.path.join(base_dir,"models")
if not os.path.isdir(os.path.join(model_dir, model_name)):
	print(f"Downloading {model_name} model...")
	gpt2.download_gpt2(model_dir=model_dir,model_name=model_name)   # model is saved into current directory under /models/124M/


In [8]:
file_name = os.path.join(base_dir,"shakespeare.txt")
if not os.path.isfile(file_name):
	url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
	data = requests.get(url)
	
	with open(file_name, 'w') as f:
		f.write(data.text)
    

In [9]:
model_name = "124M"
base_dir     = "/content/drive/My Drive/Colab Notebooks/gpt2_learning"
model_dir    = os.path.join(base_dir,"models")

In [10]:
#### FineTuning

sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
							model_dir=model_dir,
              model_name=model_name,
							checkpoint_dir=os.path.join(base_dir,"checkpoint"),
              steps=1000)   # steps is max number of training steps


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint /content/drive/My Drive/Colab Notebooks/gpt2_learning/models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Colab Notebooks/gpt2_learning/models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.75s/it]


dataset has 338025 tokens
Training...
[1 | 8.43] loss=4.07 avg=4.07
[2 | 10.69] loss=3.82 avg=3.94
[3 | 12.96] loss=3.68 avg=3.85
[4 | 15.25] loss=3.66 avg=3.81
[5 | 17.56] loss=3.73 avg=3.79
[6 | 19.86] loss=3.85 avg=3.80
[7 | 22.18] loss=3.85 avg=3.81
[8 | 24.50] loss=3.68 avg=3.79
[9 | 26.83] loss=3.79 avg=3.79
[10 | 29.17] loss=3.40 avg=3.75
[11 | 31.55] loss=3.40 avg=3.72
[12 | 33.94] loss=3.88 avg=3.73
[13 | 36.35] loss=3.48 avg=3.71
[14 | 38.78] loss=3.51 avg=3.70
[15 | 41.24] loss=3.64 avg=3.69
[16 | 43.69] loss=3.42 avg=3.67
[17 | 46.15] loss=3.55 avg=3.67
[18 | 48.64] loss=3.33 avg=3.65
[19 | 51.16] loss=3.58 avg=3.64
[20 | 53.70] loss=3.65 avg=3.64
[21 | 56.25] loss=3.38 avg=3.63
[22 | 58.80] loss=3.49 avg=3.62
[23 | 61.35] loss=3.52 avg=3.62
[24 | 63.91] loss=3.33 avg=3.60
[25 | 66.46] loss=3.60 avg=3.60
[26 | 68.99] loss=3.48 avg=3.60
[27 | 71.50] loss=3.66 avg=3.60
[28 | 73.98] loss=3.42 avg=3.59
[29 | 76.45] loss=3.18 avg=3.58
[30 | 78.89] loss=3.35 avg=3.57
[31 | 81.34]

In [11]:
gpt2.generate(sess, checkpoint_dir=os.path.join(base_dir,"checkpoint"))


Romeo, they say,
Is not much altered by yonder sun.

ROMEO:
O, they say, I know it:
Beseech you, sir, be gone.
Take him to your chapel: I will attend him.
Come, Juliet, take me to your heart. Sweet, just, and true,
I do miss thee very, very, very, very.
I do, and will gladly take thee with me.
Come, sir, away with thee: I will attend thee.

JULIET:
No, good friar, not by my troth: come, sir; away.

FRIAR LAURENCE:
Ah, sirrah, so shall I no more but sit
With you; that my loving bride may with all
Present their special thanks in pure marriage
As they did take you an hour ago.

ROMEO:
Bless me heartily on her whose nuptial kiss
Became so rank with hers!

JULIET:
I pray thee, peace: I will, Romeo.

FRIAR LAURENCE:
Beseech you, sir, on my side; I will not marry.

ROMEO:
Then let me and my partner in crime,
An hour ago, to make room for our tent,
Shall do so with much success. But, say,
We will break the knot to-morrow morning.

FRIAR LAURENCE:
Nay, I will not, sir: as it always is,
Sometime

In [12]:
gpt2.generate(sess, checkpoint_dir=os.path.join(base_dir,"checkpoint"), prefix="2015 年")


2015 年

Provost:
Where's my wife?

Provost:
Here, sir.

QUEEN ELIZABETH:
If thou'rt sure
That any man covetous or a heretic is my wife, thou'rt a traitors husband: if thou'rt a husband and a heretic is my wife, so be it; if husband and heretic not, then father.

Provost:
Well, well; marry, sir: here it is: if you be honest, you shall divorce me; if you be a heretic and a heretic, you shall divorce me; if a heretic be honest, he shall be a heretic; if a heretic be honest, he shall be a heretic; if a heretic be honest, he shall be a heretic; if a heretic be married, he shall be a heretic; if a heretic be honest, he shall be a heretic; if a heretic be married, he shall be a heretic; if a heretic be honest, he shall be a heretic; if a heretic be false, he shall be a heretic; if a heretic be lawful, he shall be a heretic; if a heretic be ta'en, he shall be a heretic; if a heretic be ta'en, he shall be a heretic; if a heretic be married, he shall be a heretic; if a heretic be married, he sha